<a href="https://colab.research.google.com/github/MajorFeijo/RNA-e-Deep-Learning-CESAR-School-2023.02/blob/main/Trabalho%20Final/Projeto_Final_da_Disciplinal_FashionMNIST_neuralnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equipe:
* Armando Feijó de Paula - afp2@cesar.school

## Data sete escolhido:
* https://github.com/zalandoresearch/fashion-mnist


# Treinamento com interface de alto nível

## Importação das bibliotecas

In [ ]:
# http://pytorch.org/
from os.path import exists

import torch

In [ ]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

## Criação da rede

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10) # aumento do numero de camadas e de reuronios da NN

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)  # usar mais uma função de ativação tipo ReLU
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net() # usar uma função de ativação tipo ReLU

## Treinamento

### Criando o objeto de treinamento

In [ ]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Avaliação

In [ ]:
# Cálculo da média e Desvio padrão do dataset FashionMNIST

def mean_std(dataset):
    # Inicializar variáveis para acumular os valores dos pixels e o número total de pixels
    mean = 0.0
    std = 0.0
    num_samples = 0

    # Iterar pelo dataset para calcular a média
    for data in dataset:
        image = data[0]
        mean += image.mean()
        num_samples += 1

    mean /= num_samples

    # Iterar pelo dataset novamente para calcular o desvio padrão
    for data in dataset:
        image = data[0]
        std += ((image - mean) ** 2).sum()

    std = torch.sqrt(std / (num_samples * 28 * 28))

    return mean, std



In [ ]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 128} # aumento do tamanho dos batches de treino
test_kwargs = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

mean, std = mean_std(datasets.FashionMNIST(
    root='../data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)) # calculo de mean e std do dataset FashionMNIST sem tratamento

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
    ])
# dataset1 = datasets.MNIST('../data', train=True, download=True,
#                    transform=transform)
# dataset2 = datasets.MNIST('../data', train=False,
#                    transform=transform)
# train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
# test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

# Load a different dataset
# new_dataset = datasets.FashionMNIST('../data', download=True, train=True, transform=transform)
train_dataset = datasets.FashionMNIST('../data', download=True, train=True, transform=transform)
test_dataset = datasets.FashionMNIST('../data', download=True, train=False, transform=transform)

# Create a new data loader for the new dataset
new_train_loader = torch.utils.data.DataLoader(train_dataset, **train_kwargs)
new_test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)

model = Net().to(device)

# optimizer = optim.Adam(model.parameters(), lr=0.01) # mudança do optmizador e diminuição do Learning rate

optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # mudança do optmizador e diminuição do Learning rate

epochs = 20 # aumento do numero de épocas
# scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) # mudança do scheduler



for epoch in range(1, epochs + 1):
    train(10, False, model, device, new_train_loader, optimizer, epoch)
    test(model, device, new_test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304282
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.139085
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.862778
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.730511
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.630469
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.569035
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.589158
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.667872
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.490683
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.665207
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.474461
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.588753
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.483361
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.364495
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.460904
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.514643
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.402487
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.494005
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.567753
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.355975
Train Epoch: 1 [

## O PROCESSO E A EXPERIÊNCIA:

## (i) Carregamento do Dataset e Manipulação de Dados

- O dataset escolhido foi o FashionMNIST e os dados foram carregados a partir de [https://github.com/zalandoresearch/fashion-mnist](https://github.com/zalandoresearch/fashion-mnist).

- A manipulação dos dados consistiu em:

  1. Transformar as imagens do dataset em tensores.
  2. Calcular a média e o desvio padrão do dataset.
  3. Normalizar as imagens usando a média e o desvio padrão dos dados.

## (ii) Definição do Modelo da Rede

- Modelo de rede neural, baseado no colab da atividade da aula 4.6 MNIST_neuralnet da disciplina Redes Neurais e Deep Learning ministrada pelo Prof. Victor Casadei: uma rede convolucional composta por duas camadas convolucionais seguidas de pooling e dropout, e três camadas totalmente conectadas com dropout entre a primeira e a segunda.
- Função de ativação: ReLU usada para introduzir não-linearidade.
- Saída: distribuição de probabilidade logarítmica sobre as 10 classes, obtida usando log-softmax.

- Fluxo de Dados:
  1. Entrada -> conv1 -> ReLU -> conv2 -> ReLU -> max_pool2d -> dropout1 -> flatten.
  2. Flatten -> fc1 -> ReLU -> dropout2 -> fc2 -> ReLU -> fc3.
  3. fc3 -> Log-Softmax -> Saída.

- O modelo se propõe a, dada uma imagem do tipo MNIST de itens de vestuário, classificá-la nas seguintes categorias:

  1. T-shirt/top (Camiseta/top)
  2. Trouser (Calça)
  3. Pullover (Pulôver)
  4. Dress (Vestido)
  5. Coat (Casaco)
  6. Sandal (Sandália)
  7. Shirt (Camisa)
  8. Sneaker (Tênis)
  9. Bag (Bolsa)
  10. Ankle boot (Bota)

## (iii) Treinamento do Modelo

### Mudanças Realizadas no Código

Melhorias foram feitas para aumentar a precisão do modelo e evitar o overfitting. O aumento do número de camadas e de neurônios da rede neural permitiu que o modelo aprendesse características mais complexas dos dados. A utilização de mais uma função de ativação tipo ReLU também ajudou a evitar o overfitting. O aumento do tamanho dos batches de treino fez com que o modelo fosse atualizado com mais dados a cada iteração, o que também ajudou a evitar o overfitting. A mudança do otimizador para Adam com uma taxa de aprendizado menor permitiu que o modelo convergisse mais rapidamente para uma solução ótima. O aumento do número de épocas propiciou ao modelo mais tempo para aprender com os dados de treino. A mudança do scheduler para StepLR com um fator de decaimento menor permitiu que a taxa de aprendizado fosse reduzida gradualmente ao longo do treinamento, o que também ajuda a evitar o overfitting. Em resumo, as melhorias foram:

- Aumento do número de camadas e de neurônios da rede neural.
- Utilização de mais uma função de ativação tipo ReLU.
- Aumento do tamanho dos batches de treino.
- Mudança do otimizador para Adam com uma taxa de aprendizado menor.
- Aumento do número de épocas.
- Mudança do scheduler para StepLR com um fator de decaimento menor.

# (iv) Acurácia do Modelo Treinado

- O modelo obteve uma acurácia de 93% no conjunto de teste. Isso significa que o modelo foi capaz de classificar corretamente 93% das imagens do conjunto de teste.
- A perda média no conjunto de teste foi de 0.2045. Isso significa que o modelo, em média, cometeu um erro de classificação de 0.2045 para cada imagem do conjunto de teste.
- Esses resultados já são bons, mas podem ser melhorados, por exemplo, treinando o modelo por mais épocas e/ou usando um otimizador diferente e/ou utilizando uma arquitetura de rede diferente.
- No geral, o modelo obteve um bom desempenho no conjunto de teste, o que indica que ele é capaz de classificar corretamente imagens do dataset FashionMNIST.

# CONCLUSÕES:
A implementação da rede neural para o dataset FashionMNIST demonstraram resultados promissores, com uma acurácia de 93% no conjunto de teste.
As melhorias realizadas, como o aumento do número de camadas e neurônios, a utilização de funções de ativação adicionais, a mudança do otimizador para Adam e ajustes no scheduler, foram fundamentais para alcançar este nível de precisão, além de mitigar o overfitting.
A capacidade do modelo de aprender características complexas dos dados foi aumentada, e a utilização de estratégias de regularização adicionais contribuiu para a estabilidade e eficácia do treinamento.
Embora o desempenho obtido seja significativo, existem oportunidades para aprimoramento, como o ajuste de hiperparâmetros e a exploração de arquiteturas de rede ainda mais complexas.
No geral, a rede desenvolvida mostra-se competente para a tarefa de classificação de itens de vestuário, destacando a importância de técnicas avançadas de treinamento e ajustes finos na obtenção de modelos robustos e precisos.

---





